In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Question Answering with Generative Models on Vertex AI


<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/question_answering.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/question_answering.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/question_answering.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

Large language models can be used for various natural language processing tasks, including question-answering (Q&A). These models are trained on a vast amount text data and can generate high-quality responses to a wide range of questions. One thing to note here is that most models have cutoff dates regarding their knowledge, and asking anything too recent might yield an incomplete, imaginative or incorrect answer (i.e. a hallucination).

This notebook covers the essentials of prompts for answering questions using a generative model. In addition, it showcases the `open domain` (knowledge available on the public internet) and `closed domain` (knowledge that is more private - typically enterprise or personal knowledge).

Learn more about prompt design in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview#prompt_structure).

### Objective

By the end of the notebook, you should be able to write prompts for the following:

* **Open domain** questions:
    * Zero-shot prompting
    * Few-shot prompting


* **Closed domain** questions:
    * Providing custom knowledge as context
    * Instruction-tune the outputs
    * Few-shot prompting

## Getting Started

### Install Vertex AI SDK

In [ ]:
!pip install google-cloud-aiplatform --upgrade --user

**Colab only:** Uncomment the following cell to restart the kernel or use the button to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top. 

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Authenticating your notebook environment
* If you are using **Colab** to run this notebook, uncomment the cell below and continue.
* If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Import libraries

**Colab only:** Uncomment the following cell to initialize the Vertex AI SDK. For Vertex AI Workbench, you don't need to run this.  

In [ ]:
# import vertexai

# PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
import pandas as pd
from vertexai.preview.language_models import TextGenerationModel

### Import models

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

## Question Answering

Question-answering capabilities require providing a prompt or a question that the model can use to generate a response. The prompt can be a few words or a few complete sentences, depending on the complexity of the question.

When creating a question-answering prompt, it is essential to be specific and provide as much context as possible. It helps the model understand the intent behind the question and generate a relevant response. For example, if you want to ask:

```
"What is the capital of France?",

then a good prompt could be:

"Please tell me the name of the city that serves as the capital of France."

```

In addition to being specific, the prompt should also be grammatically correct and free of spelling errors. It helps the model generate a response that is easy to understand and contains fewer errors or inaccuracies.

By providing specific, context-rich prompts, you can help the model understand the intent behind the question and generate accurate and relevant responses.


Below are some differences between the **open domain** and **closed domain** categories for question-answering prompts.

* **Open domain**: All questions whose answers are available online already. They can belong to any category, like history, geography, countries, politics, chemistry, etc. These include trivia or general knowledge questions, like:

```
Q: Who won the Olympic gold in swimming?
Q: Who is the President of [given country]?
Q: Who wrote [specific book]"?
```

Keep in mind the training cutoff of generative models, as questions involving information more recent than what the model was trained on might give incorrect or imaginative answers.


* **Closed domain**: If you have some internal knowledge base not available on the public internet, then those belong to the _closed domain_ category.
You can pass that "private" knowledge as context to the model. If prompted correctly, the model is more likely to answer from within the context provided and less likely to give answers beyond that from the open internet.

Consider the example of building a Q&A bot over your internal product documentation. In this case, you can pass the complete documentation to the model and prompt it only to answer based on that.

Typical prompt for **closed domain**:

```
Prompt: f""" Answer from the below context: \n\n
		   context: {your knowledge base} \n
		   question: {question specific to that knowledge base}  \n
		   answer: {to be predicted by model} \n
		"""
```

Below are some examples to understand these different types of prompts.

### Open Domain

#### Zero-shot prompting

In [ ]:
prompt = """Q: Q: My credit score is 650
The home purchase price is 500,000
What is my estimated monthly mortgage payment on a 30 year loan at today's current interest rates?
\n
            A:
         """
print(
    generation_model.predict(
        prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text
)

#### Few-shot prompting

Let's say you want to a get a short answer from the model (like only a specific name). To do so, you can leverage a few-shot prompt and provide examples to the model to illustrate the expected behavior.

In [ ]:
prompt = """My credit score is 650
The home purchase price is 500,000
What is my estimated monthly mortgage payment on a 30 year loan at today\'s current interest rates?


input: My credit score is 650
The home purchase price is 500,000
What is my estimated monthly mortgage payment on a 30 year loan at today\'s current interest rates?
output: USD 3.405


input: My credit score is 690
The home purchase price is 500,000
What is my estimated monthly mortgage payment on a 30 year loan at today\'s current interest rates?
output: USD 3.240


input: My credit score is 725
The home purchase price is 500,000
What is my estimated monthly mortgage payment on a 30 year loan at today\'s current interest rates?


output: USD 3.104


input: My credit score is 750
The home purchase price is 500,000
What is my estimated monthly mortgage payment on a 30 year loan at today\'s current interest rates?
output: """
print(
    generation_model.predict(
        prompt,
        max_output_tokens=20,
        temperature=0.1,
    ).text
)

#### Zero-shot prompting vs Few-shot prompting

Zero-shot prompting can be useful for quickly generating text for new tasks, but the quality of the generated text may be lower than that of a few-shot prompt with well-chosen examples. Few-shot prompting is typically better suited for tasks that require a high degree of specificity or domain-specific knowledge, but requires some additional thought and potentially data to set up the prompt.

### Closed Domain

#### Adding internal knowledge as context in prompts

Imagine a scenario where you would like to build a question-answering bot that takes in internal documentation and lets users ask questions about it.

In the example below, the Google Cloud Storage and content policy documentation is added to the prompt, so that the PaLM API can use that to answer subsequent questions with the provided context.

In [ ]:
context = """
To make an estimate of the monthly mortgage payment, including property taxes and\n insurance, using a mortgage calculator based on credit score, first enter loan amount \n ( The total amount you plan to borrow for your mortgage), interest rate (The annual \n interest rate on your mortgage loan) and loan term (The duration of the mortgage \n loan in years) Some mortgage calculators may have an option to input your credit score \n (Credit score: Your credit score, which can affect the interest rate offered to \n you). If available, enter your credit score to get a more accurate estimate. Your \n credit score can affect the interest rate offered by lenders.\n
"""

question = "How is high availability achieved?"

prompt = f"""Answer the question given in the contex below:
Context: {context}?\n
Question: {question} \n
Answer:
"""

print("[Prompt]")
print(prompt)

print("[Response]")
print(
    generation_model.predict(
        prompt,
    ).text
)

#### Instruction-tuning outputs

Another way to help out language models is to provide additional instructions to frame the output in the prompt. To ensure the model doesn't respond to anything outside the context, the prompt can specify that the response should be "Information not available in provided context" if that's the case.

In [ ]:
question = "PLACEHOLDER"
prompt = f"""PLACEHOLDER". \n\n
Context: {context}?\n
Question: {question} \n
Answer:
"""

print("[Prompt]")
print(prompt)

print("[Response]")
print(
    generation_model.predict(
        prompt,
        max_output_tokens=256,
        temperature=0.3,
    ).text
)

#### Few-shot prompting

In [ ]:
prompt = """
Context:
To make an estimate of the monthly mortgage payment, including property taxes and\n insurance, using a mortgage calculator based on credit score, first enter loan amount \n ( The total amount you plan to borrow for your mortgage), interest rate (The annual \n interest rate on your mortgage loan) and loan term (The duration of the mortgage \n loan in years) Some mortgage calculators may have an option to input your credit score \n (Credit score: Your credit score, which can affect the interest rate offered to \n you). If available, enter your credit score to get a more accurate estimate. Your \n credit score can affect the interest rate offered by lenders.\n
input: My credit score is 650
The home purchase price is 500,000
What is my estimated monthly mortgage payment on a 30 year loan at today\'s current interest rates?
output: USD 3.405


input: My credit score is 690
The home purchase price is 500,000
What is my estimated monthly mortgage payment on a 30 year loan at today\'s current interest rates?
output: USD 3.240


input: My credit score is 725
The home purchase price is 500,000
What is my estimated monthly mortgage payment on a 30 year loan at today\'s current interest rates?


output: USD 3.104


input: My credit score is 750
The home purchase price is 500,000

Question:What is my estimated monthly mortgage payment on a 30 year loan at today\'s current interest rates?

Answer:

"""
print(
    generation_model.predict(
        prompt,
    ).text
)

### Evaluation

You can evaluate the outputs of the question and answering task if the ground truth answers of each question are available. In zero-shot prompting, you can only use `open domain` questions. However, with `closed domain` questions, you can add context and evaluate similarly.  To showcase how that will work, start by creating a simple dataframe with questions and ground truth answers. 

In [ ]:
qa_data = {
    "question": [
        "In a website browser address bar, what does “www” stand for?",
        "Who was the first woman to win a Nobel Prize",
        "What is the name of the Earth’s largest ocean?",
    ],
    "answer_groundtruth": ["World Wide Web", "Marie Curie", "The Pacific Ocean"],
}
qa_data_df = pd.DataFrame(qa_data)
qa_data_df

Now that you have the data with questions and ground truth answers, you can call the PaLM 2 generation model to each review row using the `apply` function. Each row will use the dynamic prompt to predict the answer using the PaLM API. We will save the results in `answer_prediction` column.  


In [ ]:
def get_answer(row):
    prompt = f"""Answer the following question as precise as possible.\n\n
            question: {row}
            answer:
              """
    return generation_model.predict(
        prompt=prompt,
    ).text


qa_data_df["answer_prediction"] = qa_data_df["question"].apply(get_answer)
qa_data_df

You may want to evaluate the answers predicted by the PaLM API. However, it will be more complex than the text classification since the answers may differ from ground truth and may be presented in slightly more/fewer words. 

For example, you can observe the question "What is the name of the Earth's largest ocean?" and see that model predicted  "Pacific Ocean" when a ground truth label is "The Pacific Ocean" with the extra "The." Now, if you use the simple classification metrics, then you will consider this as a wrong prediction since original and predicted strings have a difference. However, you can see that the answer is correct since an extra "The" is causing the issue. It's a simple string comparison problem.

The solution to string comparison where both `ground_thruth` and `predicted` may have some extra or fewer letters, one approach is to use a fuzzy matching algorithm. 
Fuzzy string matching uses [Levenshtein Distance](https://en.wikipedia.org/wiki/Levenshtein_distance) to calculate the differences between two strings. 

For example, the Levenshtein distance between "kitten" and "sitting" is 3, since the following 3 edits change one into the other, and there is no way to do it with fewer than 3 edits:

* kitten → sitten (substitution of "s" for "k"),
* sitten → sittin (substitution of "i" for "e"),
* sittin → sitting (insertion of "g" at the end).


Here's another example, but this time using `fuzzywuzzy`  library, which gives us the same `Levenshtein distance` between two strings but in ratio. The ratio raw score measures the string's similarity as an int in the range [0, 100]. For two strings X and Y, the score is defined by int(round((2.0 * M / T) * 100)) where T is the total number of characters in both strings, and M is the number of matches in the two strings. 

Read more here about the [ratio formula](https://anhaidgroup.github.io/py_stringmatching/v0.3.x/Ratio.html) : 

You can see one example to understand this furhter. 
```
String1: "this is a test"
String2: "this is a test!"

Fuzz Ratio => 97  #

Fuzz Partial Ratio => 100  #Since most characters are the same and in a similar sequence, the algorithm calculates the partial ratio as 100 and ignores simple additions (new characters). 
```


First, install the package `fuzzywuzzy` and `python-Levenshtein`:

In [ ]:
!pip install -q python-Levenshtein --upgrade --user
!pip install -q fuzzywuzzy --upgrade --user

Then compute a score to perform fuzzy matching:

In [ ]:
from fuzzywuzzy import fuzz


def get_fuzzy_match(df):
    return fuzz.partial_ratio(df["answer_groundtruth"], df["answer_prediction"])


qa_data_df["match_score"] = qa_data_df.apply(get_fuzzy_match, axis=1)
qa_data_df

Now that you have the individual match score (partial), you can take the mean or average of the whole column to get a sense of overall data. 
Scores closer to 100 mean PaLM 2 can predict closer to ground truth; if the score is towards 50 or 0, it did not perform well.

In [ ]:
print(
    "the average match score of all predicted answer from PaLM 2 is : ",
    qa_data_df["match_score"].mean(),
    " %",
)

In this case, you get 100% as the mean score, even though some predictions were missing some words. That means you are very close to the ground truth, and some answers are just missing the exact verboseness of the ground truth. 